<a href="https://colab.research.google.com/github/pskshyam/Customer-Segmentation/blob/master/CustomerSegmentationUsingRFMModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Why should you care about customer segmentation? **

To deliver personalized experiences to customers, segmentation is key. It can provide insights into your customers behavior, habits, and preferences, allowing you to offer tailored marketing campaigns increasing your odds of success as well as improving your customers experience with tailored content.

**What are we going to build? **

Using transactional purchasing data, we will be able to create a 2 x 2 value matrix to create 4 customers groups. Each group will defer from the other depending on 2 dimensions: 

(1) current customer value, and

(2) potential customer value.

**What technique are we going to use? **

We are going to use the RFM model to create the required features from transactional purchasing data. The RFM model stands for:

Recency: When was the last time they purchased?<br>
Frequency: How often and for how long have they purchased?<br>
Monetary Value/Sales: How much have they purchased?<br>

It’s usually used to identify the Highest Value Customer at the intersection of each 3 questions. To build the 2 x 2 matrix we will only use the R & the M from RFM.

![alt text](https://cdn-images-1.medium.com/max/800/1*7JfiT-GBi4R501svnVrQFA.png)

**What data are we using?** 

We are going to use the purchased sample data set provided by Tableau also known as “Global Superstore”. It is often used for forecasting and time series analysis. It contains more than 1500 different customers and 4 years of purchase data. Since we are doing a behavioral segmentation and not a demographic segmentation, we will remove some potential demographic bias by filtering only on the B2C segment (consumer) and the USA country.

In [0]:
#Install necessary libraries
import matplotlib as plt
import numpy as np
%matplotlib inline  
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

In [3]:
#Install xlrd package for reading excel files
!pip install xlrd

    100% |████████████████████████████████| 112kB 2.7MB/s 


In [4]:
#Load the dataset
url = 'https://github.com/tristanga/Data-Analysis/raw/master/Global%20Superstore.xls'
df = pd.read_excel(url)
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,2013-02-05,2013-02-07,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,2013-10-17,2013-10-18,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,2013-01-28,2013-01-30,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,2013-11-05,2013-11-06,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [5]:
#Filter the dataset for Consumer segment and USA country
df = df[(df.Segment == 'Consumer') & (df.Country == 'United States')]
df.head()

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,2012-07-31,2012-07-31,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
9,40936,CA-2012-116638,2012-01-28,2012-01-31,Second Class,JH-15985,Joseph Holt,Consumer,Concord,North Carolina,...,FUR-TA-10000198,Furniture,Tables,Chromcraft Bull-Nose Wood Oval Conference Tabl...,4297.644,13,0.4,-1862.3124,865.74,Critical
21,31784,CA-2011-154627,2011-10-29,2011-10-31,First Class,SA-20830,Sue Ann Reed,Consumer,Chicago,Illinois,...,TEC-PH-10001363,Technology,Phones,Apple iPhone 5S,2735.952,6,0.2,341.9940,752.51,High
32,32735,CA-2012-139731,2012-10-15,2012-10-15,Same Day,JE-15745,Joel Eaton,Consumer,Amarillo,Texas,...,FUR-CH-10002024,Furniture,Chairs,HON 5400 Series Task Chairs for Big and Tall,2453.430,5,0.3,-350.4900,690.42,High
34,32543,CA-2011-168494,2011-12-12,2011-12-14,Second Class,NP-18700,Nora Preis,Consumer,Fresno,California,...,FUR-TA-10003473,Furniture,Tables,Bretford Rectangular Conference Table Tops,3610.848,12,0.2,135.4068,683.12,High


**What approach are we taking?**

Step 0: Load, filter, clean, and aggregate the data at the customer level,

Step 1: Create RFM Features for each customers,

Step 2: To automate the segmentation we will use the 80% quantile for Recency and Monetary (we could have also used k-mean clustering or leveraged business knowledge to create buckets — for example global superstore business users consider an active customer as someone whose last order is less than 100 days old),

Step 3: Calculate RM score and sort customers,

Step 4: Visualize the Value Matrix and explore some key numbers.

**Step 1: Create RFM Features for each customer**

In [6]:
df_RFM = df.groupby('Customer ID').agg({'Order Date': lambda y: (df['Order Date'].max().date() - y.max().date()).days,
                                        'Order ID': lambda y: len(y.unique()),  
                                        'Sales': lambda y: round(y.sum(),2)})
df_RFM.columns = ['Recency', 'Frequency', 'Monetary']
df_RFM = df_RFM.sort_values('Monetary', ascending=False)
df_RFM.head()

,Recency,Frequency,Monetary
Customer ID,,,
RB-19360,96,6,15117.34
AB-10105,41,10,14473.57
KL-16645,47,12,14175.23
SC-20095,349,9,14142.33
HL-15040,43,6,12873.30


**Step 2: To automate the segmentation we will use 80% quantile for Recency and Monetary**

In [7]:
# We will use the 80% quantile for each feature
quantiles = df_RFM.quantile(q=[0.8])
print(quantiles)

     Recency  Frequency  Monetary
0.8    222.2        8.0   4070.17


In [8]:
df_RFM['R']=np.where(df_RFM['Recency']<=int(quantiles.Recency.values), 2, 1)
df_RFM['F']=np.where(df_RFM['Frequency']>=int(quantiles.Frequency.values), 2, 1)
df_RFM['M']=np.where(df_RFM['Monetary']>=int(quantiles.Monetary.values), 2, 1)
df_RFM.head()

,Recency,Frequency,Monetary,R,F,M
Customer ID,,,,,,
RB-19360,96,6,15117.34,2,1,2
AB-10105,41,10,14473.57,2,2,2
KL-16645,47,12,14175.23,2,2,2
SC-20095,349,9,14142.33,1,2,2
HL-15040,43,6,12873.30,2,1,2


**Step 3: Calculate RFM score and sort customers**

In [9]:
# To do the 2 x 2 matrix we will only use Recency & Monetary
df_RFM['RMScore'] = df_RFM.M.map(str)+df_RFM.R.map(str)
df_RFM.head()

,Recency,Frequency,Monetary,R,F,M,RMScore
Customer ID,,,,,,,
RB-19360,96,6,15117.34,2,1,2,22
AB-10105,41,10,14473.57,2,2,2,22
KL-16645,47,12,14175.23,2,2,2,22
SC-20095,349,9,14142.33,1,2,2,21
HL-15040,43,6,12873.30,2,1,2,22


In [10]:
df_RFM = df_RFM.reset_index()
df_RFM.head()

,Customer ID,Recency,Frequency,Monetary,R,F,M,RMScore
0,RB-19360,96,6,15117.34,2,1,2,22
1,AB-10105,41,10,14473.57,2,2,2,22
2,KL-16645,47,12,14175.23,2,2,2,22
3,SC-20095,349,9,14142.33,1,2,2,21
4,HL-15040,43,6,12873.30,2,1,2,22


In [12]:
df_RFM_SUM = df_RFM.groupby('RMScore').agg({'Customer ID': lambda y: len(y.unique()),
                                        'Frequency': lambda y: round(y.mean(),0),
                                        'Recency': lambda y: round(y.mean(),0),
                                        'R': lambda y: round(y.mean(),0),
                                        'M': lambda y: round(y.mean(),0),
                                        'Monetary': lambda y: round(y.mean(),0)})
df_RFM_SUM.head()

,Customer ID,Frequency,Recency,R,M,Monetary
RMScore,,,,,,
11,73,4,440,1,1,1526.0
12,254,6,70,2,1,1902.0
21,9,7,425,1,2,8564.0
22,73,8,59,2,2,6711.0


In [18]:
df_RFM_SUM = df_RFM_SUM.reset_index()
df_RFM_SUM = df_RFM_SUM.sort_values('RMScore', ascending=False)
df_RFM_SUM.head()

,RMScore,Customer ID,Frequency,Recency,R,M,Monetary
3,22,73,8,59,2,2,6711.0
2,21,9,7,425,1,2,8564.0
1,12,254,6,70,2,1,1902.0
0,11,73,4,440,1,1,1526.0


**Step 4: Visualize the Value Matrix and explore some key numbers**

In [22]:
# 1) Average Monetary Matrix
df_RFM_M = df_RFM_SUM.pivot(index='M', columns='R', values='Monetary')
df_RFM_M= df_RFM_M.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
df_RFM_M

R,1,2
M,,
2,8564.0,6711.0
1,1526.0,1902.0


In [23]:
# 2) Number of Customer Matrix
df_RFM_C = df_RFM_SUM.pivot(index='M', columns='R', values='Customer ID')
df_RFM_C= df_RFM_C.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
df_RFM_C

R,1,2
M,,
2,9,73
1,73,254


In [24]:
# 3) Recency Matrix
df_RFM_R = df_RFM_SUM.pivot(index='M', columns='R', values='Recency')
df_RFM_R= df_RFM_C.reset_index().sort_values(['M'], ascending = False).set_index(['M'])
df_RFM_R

R,1,2
M,,
2,9,73
1,73,254


![alt text](https://cdn-images-1.medium.com/max/800/1*3VPCRFGYzxHmMBepRKqouQ.png)

![alt text](https://cdn-images-1.medium.com/max/800/1*zESqrQ1e6Tm5Uwu23PFtGA.png)

**Some takeaways / quick wins with very simple sales & marketing tactics examples?**

* There are few customers in the “Disengaged” bucket and they have an average revenue higher than the “Star” bucket. Since there are very few of them, it should be easy to partner with the business to understand what happened at the customer level. Based on the analysis, there might be a simple quick win: reactivate few of them with a phone call or meeting to hopefully move them back to the “Star” bucket (e.g. engaged customers).


* The average last order from the “Light” bucket is very old (more than 1 year vs. 60-70 days for ‘engaged’ customers). Launching a simple reactivation campaign with a coupon might be an initiative that could lead to some new orders and help some of these customers move to the “New” bucket (e.g. engaged customers).

![alt text](https://cdn-images-1.medium.com/max/800/1*wZY7pxdnVgv-34v7gRQ9jw.png)